# Визуализация обучения
На некоторых этапах бывает полезно для отладки изобразить, как изменяется точность, как меняются веса слоёв, что поступает на вход. В этом может помочь tensorboard.

Что нам понадобится установить: 
* tensorflow
* tensorboard
* mxboard

Будем смотреть на примере предыдущего ноутбука с набором данных MNIST.

In [8]:
import mxnet as mx
from mxboard import SummaryWriter #то, что понадобится для визуализации
import os #потребуется для сохранения логов

MXNet для отладки предоставляет callback функции, которые можно вызывать под конец эпохи, при валидации, под конец пакета данных. 

Создадим три функции:
* metrics_check будет в конце каждой валидации сохранять число-метрику, которую потом мы отобразим на графике
* first_epoch_images покажет, какие изображения передавались на первой эпохе
* epoch_end_results будет сохранять гистограммы, как меняются веса и отклонения слоёв в течение всего процесса обучения

In [2]:
def metrics_check(*args):
    '''
    Примерный список входных вариантов такой:
    (номер_эпохи, номер_пакета, метрики, заголовки={
    пакет данных и т.д.
    })
    Для более подробной информации можно вывести аргументы функцикей print
    '''
    args = args[0] #получаем кортеж аргументов
    epoch = args[0] #получаем номер текущей эпохи
    metrics = args[2] #извлекаем метрики
    for name, value in metrics.get_name_value(): #возвращает лист кортежей вида (название, значение)
        #функция создаёт график, шагом построения точек выбираем эпоху
        summary_writer.add_scalar(tag=name,
                                  value=value,
                                  global_step=epoch)
        
        
def first_epoch_images(*args):
    args = args[0]
    epoch = args[0]
    if epoch != 0: #если эпоха не первая, то сохранять изображения не нужно
        return
    num_batch = args[1] #номер пакета в первой эпохе будет нашим шагом сохранения изображений
    data = args[3]['data_batch'].data #достаём из заголовков
    summary_writer.add_image(tag='images_per_epoch',
                             image=data[0]/256, #функция принимает матрицы, где значения от 0 до 1
                             global_step=num_batch)
    
    
def epoch_end_results(*args):
    epoch = args[0]
    layers = args[2] #получение словаря вида {слой_вес/отклонение: значения}
    for key, value in layers.items():
        summary_writer.add_histogram(tag=key,
                                     values=value,
                                     global_step=epoch,
                                     bins=1000)

Так же для простоты кода добавим две функции, перезагружающие параметры для каждого из входных значений скорости обучения (0.1, 0.01, 0.001)

In [24]:
def reset_params(learning_rate):
    train_iter.reset()
    valid_iter.reset()
    summary_writer.add_graph(mlp) #сохраним граф наших слоёв
    mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
    mod.init_params(force_init=True,
                    initializer=mx.initializer.Xavier(magnitude=2.))
    mod.init_optimizer(force_init=True,
                       optimizer='sgd',
                       optimizer_params={'learning_rate': learning_rate})


def start_train(epochs):
    mod.fit(train_data=train_iter,
            eval_data=valid_iter,
            eval_metric=['loss', 'acc'], #нас будут интересовать потери на каждой эпохе и точность
            eval_end_callback=metrics_check, #функциям callback можно передать список функций для вызова
                                             #данная вызовется под конец валидации
            batch_end_callback=first_epoch_images, #под конец пакета
            epoch_end_callback=epoch_end_results, #под конец эпохи
            num_epoch=epochs)

Проинициализируем слои

In [25]:
data = mx.sym.Variable('data')
data = mx.sym.Flatten(data=data)
fc1  = mx.sym.FullyConnected(data=data, name='fc1', num_hidden=128)
act1 = mx.sym.Activation(data=fc1, name='relu1', act_type="relu")
fc2  = mx.sym.FullyConnected(data=act1, name='fc2', num_hidden=10)
mlp  = mx.sym.SoftmaxOutput(data=fc2, name='softmax')
mod = mx.mod.Module(mlp) #mlp - multilayer perceptron

In [26]:
train_iter = mx.io.ImageRecordIter(path_imglist='./mnist_train.lst', 
                                   path_imgrec='./mnist_train.rec',
                                   data_shape=(1, 28, 28),
                                   batch_size=100)

valid_iter = mx.io.ImageRecordIter(path_imglist='./mnist_test.lst', 
                                   path_imgrec='./mnist_test.rec',
                                   data_shape=(1, 28, 28),
                                   batch_size=100)

Проведём тот же эксперимент, но для большего количества эпох

In [27]:
learning_rate = [0.1, 0.01, 0.001]
num_of_epochs = [20, 20, 25]
#домашняя директория, например, куда будут сохраняться логи для последующей визуализации
logdir = os.path.join(os.environ['HOME'] + '/logs/learning_rate=')
for lr, epochs in zip(learning_rate, num_of_epochs):
    #создадим для каждого значения lr отдельную папку логов
    #и в конце каждого обучения будем закрывать summary_writer благодаря with
    with SummaryWriter(logdir + str(lr), flush_secs=3) as summary_writer:
        reset_params(lr)
        start_train(epochs)

INFO:mxboard.event_file_writer:successfully opened events file: /home/gleb/logs/learning_rate=0.1/events.out.tfevents.1531735715.glkv
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
/home/gleb/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:mxboard.event_file_writer:wrote 40 events to disk
INFO:mxboard.event_file_writer:wrote 45 events to disk
INFO:mxboard.event_file_writer:wrote 51 events to disk
INFO:mxboard.event_file_writer:wrote 52 events to disk
INFO:mxboard.event_file_writer:wrote 43 events to disk
INFO:mxboard.event_file_writer:wrote 47 events to disk
INFO:mxboard.event_file_writer:wrote 49 events to disk
INFO:mxboard.event_file_writer:wrote 26 events to disk
INFO:mxboard.event_file_writer:wrote 14 events to disk
INFO:mxboard.event_file_writer:wrote 

Далее, в консоли укажем папку logs для запуска tensorboard

In [32]:
tens_logs = os.path.join(os.environ["HOME"], 'logs/')
!tensorboard --logdir=$tens_logs #через ! команды отправляются в командную строку

TensorBoard 1.9.0 at http://glkv:6006 (Press CTRL+C to quit)
^C


Посмотрим на результаты.
Точность не растёт лишь у первого эксперимента. Функция потерь также отмечает изменения.
<img src="images/pic1.png">
На первом слое отклонения в первом случае(lr=0.1) смещены от нуля примерно на 2.10, а веса - на -376. Т.е. сеть не может попасть с таким большим шагом обучения в нужные веса.
<img src="images/pic2.png">
На третьей картине так же, как и на второй, при lr=0.1 веса имеют слишком большое значение(333), что не позволяет нейросети делать верные выводы о предоставляемых данных.
<img src="images/pic3.png">

# Сохранение обученной нейросети

In [30]:
#будем делать чекпоинт после каждой эпохи(можно задать самим)
model_prefix = 'mnist_net'
checkpoint = mx.callback.do_checkpoint(model_prefix)

In [31]:
lr = 0.0001 #для интереса попробуем ещё меньшую скорость обучения
epochs = 25
with SummaryWriter(logdir + str(lr), flush_secs=3) as summary_writer:
    reset_params(lr)
    mod.fit(train_data=train_iter,
            eval_data=valid_iter,
            eval_metric=['loss', 'acc'],
            eval_end_callback=metrics_check,
            batch_end_callback=first_epoch_images,
            epoch_end_callback=[epoch_end_results, checkpoint], #передадим лист с функциями, где есть чекпоинт
            num_epoch=epochs)

INFO:mxboard.event_file_writer:successfully opened events file: /home/gleb/logs/learning_rate=0.0001/events.out.tfevents.1531737737.glkv
/home/gleb/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 41 events to disk
INFO:mxboard.event_file_writer:wrote 38 events to disk
INFO:mxboard.event_file_writer:wrote 45 events to disk
INFO:mxboard.event_file_writer:wrote 44 events to disk
INFO:mxboard.event_file_writer:wrote 38 events to disk
INFO:mxboard.event_file_writer:wrote 36 events to disk
INFO:mxboard.event_file_writer:wrote 37 events to disk
INFO:mxboard.event_file_writer:wrote 38 events to disk
INFO:mxboard.event_file_writer:wrote 30 events to disk
INFO:mxboard.event_file_writer:wro

В папке с ноутбуком будут храниться сохранённые параметры модели для каждой эпохи.

In [43]:
#загрузка состояния модели для третьей эпохи
sym, arg_params, aux_params = mx.model.load_checkpoint(model_prefix, 3) 

#print(arg_params) #вывод параметров, передаются веса и отклонения на загруженной эпохе
mod.set_params(arg_params, aux_params)
#продолжим обучение с третьей эпохи

In [44]:
mod = mx.mod.Module(symbol=sym) #присвоим новый Символ модели
# assign the loaded parameters to the module
mod.fit(train_iter,
        num_epoch=25,
        begin_epoch=3)